In [1]:
import pandas as pd

In [2]:
# Read the sales data into a pandas dataframe
df = pd.ExcelFile('block_sales_2021.xlsx')
df = df.parse('Sheet1')

In [3]:
# Let us have a look at the dataframe before storing it into the database we are to create.
df.head()

,order id,date,order type,unit price,order quantity,order amount,buyer first name,buyer surname,buyer gender,buyer type
0,20210001,2021-01-04,9 inches,300,301,90300,Ekemini,Idongesit,M,Contractor
1,20210002,2021-01-04,9 inches,300,727,218100,Nnenna,Anayo,F,Supplier
2,20210003,2021-01-04,9 inches,300,775,232500,Adamu,Kabiru,M,Building Owner
3,20210004,2021-01-05,6 inches,250,177,44250,Evans,Chika,M,Building Owner
4,20210005,2021-01-05,4 inches,200,157,31400,Nnenna,Anayo,F,Supplier


In [4]:
# Let us remove spaces from column names and then proceed with the database creation.
df = df.rename(columns = {'order id':'order_id', 'order type':'order_type', 'unit price':'unit_price','order quantity':
                          'order_quantity', 'order amount':'order_amount', 'buyer_first_name':'buyer_first_name',
                         'buyer_surname':'buyer_surname', 'buyer gender':'buyer_gender', 'buyer type':'buyer_type'})
df.head(2)

,order_id,date,order_type,unit_price,order_quantity,order_amount,buyer first name,buyer surname,buyer_gender,buyer_type
0,20210001,2021-01-04,9 inches,300,301,90300,Ekemini,Idongesit,M,Contractor
1,20210002,2021-01-04,9 inches,300,727,218100,Nnenna,Anayo,F,Supplier


In [5]:
from sqlalchemy import create_engine
%load_ext sql

# Create a SQLite database
engine  = create_engine("sqlite:///block_sales_2021.db")

# Store the excel data into the database
df.to_sql('2021_orders', engine, index=False)

In [6]:
%sql sqlite:///block_sales_2021.db

In [7]:
%%sql
-- Lets check few rows of the 2021_orders table
select * from '2021_orders' limit 20

 * sqlite:///block_sales_2021.db
Done.


order_id,date,order_type,unit_price,order_quantity,order_amount,buyer first name,buyer surname,buyer_gender,buyer_type
20210001,2021-01-04 00:00:00.000000,9 inches,300,301,90300,Ekemini,Idongesit,M,Contractor
20210002,2021-01-04 00:00:00.000000,9 inches,300,727,218100,Nnenna,Anayo,F,Supplier
20210003,2021-01-04 00:00:00.000000,9 inches,300,775,232500,Adamu,Kabiru,M,Building Owner
20210004,2021-01-05 00:00:00.000000,6 inches,250,177,44250,Evans,Chika,M,Building Owner
20210005,2021-01-05 00:00:00.000000,4 inches,200,157,31400,Nnenna,Anayo,F,Supplier
20210006,2021-01-05 00:00:00.000000,6 inches,250,150,37500,Adetola,Gboyega,M,Building Owner
20210007,2021-01-05 00:00:00.000000,9 inches,300,736,220800,Sambi,Mark,M,Supplier
20210008,2021-01-05 00:00:00.000000,9 inches,300,389,116700,Gabriella,Fred,F,Engineer
20210009,2021-01-06 00:00:00.000000,4 inches,200,143,28600,Evans,Chika,M,Building Owner
20210010,2021-01-06 00:00:00.000000,6 inches,250,75,18750,Adun,Olayiwola,F,Engineer


### CONVERSION OF DATABASE TO 1NF

- We noticed some rows contain multiple records separated by **","**. We will separate such rows and make sure each row of the table contain a single record.

- Secondly, We will make sure all entries in each column contain the same data type. We will also ensure all the columns of the table have the appropriate data types.

- Lastly, we will ensure each row is uniquely identifiable by adding a primary key column to the table.

##### ENSURING THERE IS NO ROW WITH MULTIPLE RECORDS

In [8]:
%%sql
select * from '2021_orders' where order_type like '%,%' limit 5

 * sqlite:///block_sales_2021.db
Done.


order_id,date,order_type,unit_price,order_quantity,order_amount,buyer first name,buyer surname,buyer_gender,buyer_type
20210012,2021-01-07 00:00:00.000000,"6 inches, 4 inches","250, 200","248, 52",72400,Azuka,Chizoba,M,Engineer
20210034,2021-01-18 00:00:00.000000,"9 inches, 4 inches","300, 200","179, 100",73700,Saheed,Salaam,M,Supplier
20210079,2021-02-09 00:00:00.000000,"4 inches, 6 inches, 9 inches","200, 250, 300","31, 80, 70",68200,Sadio,Chiekh,M,Contractor
20210142,2021-03-29 00:00:00.000000,"6 inches, 4 inches","250, 200","242, 20",64500,Victor,Aki,M,Contractor
20210198,2021-04-09 00:00:00.000000,"6 inches, 9 inches","250, 300","245, 140",103250,Adun,Olayiwola,F,Engineer
